In [13]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableMap

# 初始化 ChatOpenAI 模型，指定使用的模型为 'gpt-4o-mini'
llm = ChatOpenAI(
    model="gpt-4o-mini",
    openai_api_base="https://api.gptsapi.net/v1",
    openai_api_key="sk-XPp5a40638eec6f1ce1c0333e36bf6305e53c8ea95ccTUXc"
)

# 1.开始节点================================
# 创建一个开始节点，用于传递初始输入
start = RunnablePassthrough()

# 2.小模型节点================================
# 模拟指标查询接口
def mock_indicator_api(input_text):
    # 模拟指标数据库
    indicators = {
        "营业收入": 0.95,
        "退货及时率": 0.78,
        "客户满意度": 0.85,
        "库存周转率": 0.72,
        "毛利率": 0.88
    }
    
    # 构造返回结果
    result = {
        "data": {
            "indicatorResult": []
        },
        "description": "success", 
        "errorCode": 0,
        "requestId": "mock-request-id"
    }
    
    # 查找最匹配的指标
    matched = False
    for name, score in indicators.items():
        if input_text in name:
            result["data"]["indicatorResult"].append({
                "指标名称": name,
                "相似度": score
            })
            matched = True
            break
    
    # 如果没找到匹配项,返回空结果
    if not matched:
        result["data"]["indicatorResult"] = []
        
    return result

# 3.参数提取================================
# 创建提示词模板
param_extract_prompt = ChatPromptTemplate.from_template("""
你是一个数据分析专家，擅长对用户自然语言进行解析，识别用户问题中的统计维度和筛选条件，以便获取更准确的指标数据。

技能: 数据分析、指标识别、维度匹配、逻辑推理。

目标: 根据用户问题从背景信息中选出与问题最相关的维度，并从用户问题中识别出其中的统计维度（group by）。

背景信息:
指标：{indicator}

约束条件:
1. 所有的思考和输出都严格基于给定的背景信息，不得超出范围，不得伪造或猜测内容。
2. 必须严格按照以下JSON格式输出，禁止附加任何其他内容。
3. 所有指标名称和维度名称都必须来自你选定的指标。
4. 牢记！当问题中的维度有具体维度值时，禁止将该维度加入统计维度。

输出格式:
{{"指标名称":"XXX","统计维度名称":[{{"维度名称":"XXX"}}],"排序维度名称":"XXX","排序方式":"排序方式(1-升序，2-降序，0-不需要排序)","结果限制条数":"结果限制条数(数字，1，2，3...)","卡片类型":"XXX(NumberCard/Chart)","图表类型":"XXX(bar/line/pie)"}}

若通过背景信息的指标无法回答用户的问题或用户的问题与指标查询无关，不要伪造内容，返回指标名称为空，按以下JSON格式返回，不要附加任何其他内容：
{{"指标名称":""}}

如果统计维度为时间维度，则"统计维度名称"节点按照：{{"维度名称":"XXX","时间粒度":"year/quarter/month/day"}} 格式输出。注意！仅统计维度为时间维度时增加"时间粒度"节点，非时间维度不增加"时间粒度"节点。

用户问题: {question}
""")

# 创建参数提取链
param_extract_chain = (
    RunnableMap({
        "indicator": lambda x: x["indicator"],
        "question": lambda x: x["question"]
    })
    | param_extract_prompt
    | llm
    | StrOutputParser()
)

# 测试数据
test_questions = [
    "按照部门统计员工数量",
    "查看2023年各月份的销售额", 
    "统计不同职级的平均工资，并按工资降序排列前5名",
    "分析各区域的客户满意度"
]

test_indicator = {
    "员工数量": {
        "维度": ["部门", "职级", "入职年份"],
        "时间维度": ["入职日期"]
    },
    "销售额": {
        "维度": ["产品", "区域", "销售渠道"],
        "时间维度": ["销售日期"]
    },
    "平均工资": {
        "维度": ["部门", "职级", "工龄"],
        "时间维度": ["统计月份"]
    },
    "客户满意度": {
        "维度": ["区域", "产品类型", "服务类型"],
        "时间维度": ["评价日期"]
    }
}

# 测试参数提取链
async def test_param_extract():
    for question in test_questions:
        print(f"\n测试问题: {question}")
        result = await param_extract_chain.ainvoke({
            "indicator": test_indicator,
            "question": question
        })
        print(f"提取结果: {result}")

await test_param_extract()



测试问题: 按照部门统计员工数量
提取结果: {"指标名称":"员工数量","统计维度名称":[{"维度名称":"部门"}],"排序维度名称":"部门","排序方式":"0","结果限制条数":"0","卡片类型":"NumberCard","图表类型":"bar"}

测试问题: 查看2023年各月份的销售额
提取结果: {"指标名称":"销售额","统计维度名称":[{"维度名称":"销售日期","时间粒度":"month"}],"排序维度名称":"销售日期","排序方式":"2","结果限制条数":"12","卡片类型":"Chart","图表类型":"bar"}

测试问题: 统计不同职级的平均工资，并按工资降序排列前5名
提取结果: {"指标名称":"平均工资","统计维度名称":[{"维度名称":"职级"}],"排序维度名称":"平均工资","排序方式":"2","结果限制条数":"5","卡片类型":"NumberCard","图表类型":"bar"}

测试问题: 分析各区域的客户满意度
提取结果: {"指标名称":"客户满意度","统计维度名称":[{"维度名称":"区域"}],"排序维度名称":"区域","排序方式":"0","结果限制条数":"1","卡片类型":"Chart","图表类型":"bar"}


In [15]:
# 构建提取筛选条件的提示词
filter_extract_prompt = ChatPromptTemplate.from_template("""你是一个数据分析助手。请帮我从用户的指标查询问题中识别筛选过滤条件。

目标：从用户的指标查询问题中识别用户问题中的筛选过滤条件(按某维度分组不属于)，并结合指标定义为筛选条件匹配可能的筛选项。

技能: 数据分析、指标理解、筛选条件匹配、逻辑推理。

约束：
1. 返回结果中的<潜在维度或指标>禁止超出<背景信息>中<指标定义>包含的范围。禁止对<潜在维度或指标>内容进行伪造。
2. 若筛选值为日期，请转换为以下格式：YYYY-MM-DD~YYYY-MM-DD，当时间点只有一个的时候(例如本日)也必须正确转化为该格式。根据当前时间：2024-01-01读取当前的年、月、日。
情况1：当用户问题涉及时间段但未明确起止时间时，默认其起止时间为本年度第一天到本年度最后一天。
情况2：当用户问题有指定月份为时间但未指定起始年份时，默认其为当前系统时间的年份。
情况3：当用户问题有指定日期为时间但未指定起始年份或月份时，默认其为当前系统时间的年份和月份。
情况4：当用户问题明确了起止时间的年、月、日时，直接按照用户的问题返回时间范围。
以上4种情况其格式都为字符串格式。
3. 返回结果中的<待搜索的筛选项>中每个对象有3个Key值，分别为<潜在维度或指标>、<筛选符号>和<筛选值>。
4. 其中，<筛选符号>的范围仅有两种，分别为"INCLUDE" #表示字符串类型的等于或匹配，"EQUAL"#表示数值类型的相等。你所选择的范围不能超出这两者，当这两种符号不能满足过滤需求时(例如分组统计)，请将其设置为"OTHER"。

输出格式:
{
  "指标名称": "XX", 
  "时间筛选类型":[{"筛选值":"XX","潜在维度或指标":["XX","...","XX"]},...,{"筛选值":"XX","潜在维度或指标":["XX","...","XX"]}],
  "待搜索的筛选项": [{"筛选符号名称":"XX","筛选值": ["XX","...","XX"],"潜在维度或指标": ["XX","...","XX"]}, {"筛选符号名称":"XX","筛选值": ["XX","...","XX"],"潜在维度或指标": ["XX","...","XX"]}]
}

背景信息:
指标定义: {indicator}

用户问题: {question}

请按照要求的格式提取筛选条件。""")

# 构建筛选条件提取链
filter_extract_chain = (
    {
        "indicator": lambda x: x["indicator"],
        "question": lambda x: x["question"]
    }
    | filter_extract_prompt
    | llm
    | StrOutputParser()
)

# 测试筛选条件提取链
async def test_filter_extract():
    for question in test_questions:
        print(f"\n测试问题: {question}")
        result = await filter_extract_chain.ainvoke({
            "\n  \"指标名称\"": test_indicator,
            "indicator": test_indicator,
            "question": question
        })
        print(f"提取结果: {result}")

await test_filter_extract()



测试问题: 按照部门统计员工数量


KeyError: 'Input to ChatPromptTemplate is missing variables {\'\\n  "指标名称"\'}.  Expected: [\'\\n  "指标名称"\', \'indicator\', \'question\'] Received: [\'indicator\', \'question\']\nNote: if you intended {\n  "指标名称"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\n  "指标名称"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '